In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Produce knn boundary graphs
from sklearn.inspection import DecisionBoundaryDisplay
import seaborn as sns

# Import datatypes for parameter type specification
from pandas.core import frame
from sklearn.preprocessing._label import LabelEncoder

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: seaborn in o:\onedrive\documents\d2l fall 2022\infs 768 - predictive analytics\env-infs768\lib\site-packages (0.12.1)

Requirement already satisfied: pandas>=0.25 in o:\onedrive\documents\d2l fall 2022\infs 768 - predictive analytics\env-infs768\lib\site-packages (from seaborn) (1.5.0)

Requirement already satisfied: matplotlib!=3.6.1,>=3.1 in o:\onedrive\documents\d2l fall 2022\infs 768 - predictive analytics\env-infs768\lib\site-packages (from seaborn) (3.6.0)

Requirement already satisfied: numpy>=1.17 in o:\onedrive\documents\d2l fall 2022\infs 768 - predictive analytics\env-infs768\lib\site-packages (from seaborn) (1.23.3)

...

Requirement already satisfied: python-dateutil>=2.7 in o:\onedrive\documents\d2l fall 2022\infs 768 - predictive analytics\env-infs768\lib\site-packages (from matplotlib!=3.6.1,>=3.1->seaborn) (2.8.2)

Requirement already satisfied: pillow>=6.2.0 in o:\onedrive\documents\d2l fall 2022\infs 768 - predictive analytics\env-infs768\lib\site-packages (from matplotlib!=3.6.1,>=3.1->seaborn) (9.2.0)

Requirement already satisfied: cycler>=0.10 in o:\onedrive\documents\d2l fall 2022\infs 768 - predictive analytics\env-infs768\lib\site-packages (from matplotlib!=3.6.1,>=3.1->seaborn) (0.11.0)

Requirement already satisfied: pytz>=2020.1 in o:\onedrive\documents\d2l fall 2022\infs 768 - predictive analytics\env-infs768\lib\site-packages (from pandas>=0.25->seaborn) (2022.4)

Requirement already satisfied: six>=1.5 in o:\onedrive\documents\d2l fall 2022\infs 768 - predictive analytics\env-infs768\lib\site-packages (from python-dateutil>=2.7->matplotlib!=3.6.1,>=3.1->seaborn) (1.16.0)

In [ ]:
# read scaled data for non-target attributes
X = pd.read_csv("X.csv")
MIN = pd.read_csv("MIN.csv")
MAX = pd.read_csv("MAX.csv")

# read the target attribute
y = pd.read_csv("y.csv")
display(y)

In [ ]:
# encode a single column from a dataframe
def encode_y(y:frame.DataFrame) -> dict:
    y = y.squeeze()
    labEnc_y = LabelEncoder()
    labEnc_y = labEnc_y.fit(y)
    y = pd.DataFrame(labEnc_y.transform(y), columns=[y.name])
    return dict(y=y,encoder=labEnc_y)

def unencode_y(y:frame.DataFrame, encoder:LabelEncoder):
    # use sklearn.preprocessing.LabelEncoder.inverse_transform() 
    # to revert the encoded data
    y = y.squeeze()
    return(pd.DataFrame(encoder.inverse_transform(y), columns=[y.name]))

# study interactions

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
encode_yItem = encode_y(y_train)
labels_train = encode_yItem.get("y")
encoder = encode_yItem.get("encoder")

In [ ]:
display(y_train.head())
pd.plotting.scatter_matrix(X_train,
                           c=labels_train.squeeze(),
                           hist_kwds={'bins':20},
                           figsize=(15,15))
plt.show()
print(type(y_train), type(y_test))

# Plot a 2d knn as an example

### Select two attributes for the KNN model

In [ ]:
names = ["mpg", "price"]
X2d = X_train.loc[:, [names[0],names[1]]]
y_train = np.ravel(y_train)
print(y_train)


### Build the model from the training set
- show boundaries for the trained model.
- points from the training set are labeled based on the car models

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1, weights="distance")
knn.fit(X2d, y_train)
_, ax = plt.subplots()

DecisionBoundaryDisplay.from_estimator(
    knn,
    X2d,
    ax=ax,
    response_method="predict",
    plot_method="pcolormesh",
    xlabel=names[0],
    ylabel=names[1],
    shading="auto",
)

# Plot also the training points
sns.scatterplot(
    x=X2d[names[0]],
    y=X2d[names[1]],
    hue=y_train,
    alpha=1.0,
    edgecolor="black",
)
plt.show()


# Assess accuracy for the sample KNN model

In [ ]:
arr = np.array([])
y_test = y_test.squeeze()
for n in range(1,21):
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(X_train, y_train)
    y_predict = knn.predict(X_test)
    m = np.mean(y_predict == y_test)
    arr = np.append(arr, np.array(m))
plt.plot(range(1,arr.size+1), arr)
plt.show()


In [ ]:
misclassified = y_test[y_test != y_predict]
names = misclassified.unique()
sum = 0
error_counts = np.array([])
name_counts = np.array([])
for name in names:
    errors = misclassified[misclassified == name].shape[0]
    sum += errors
    error_counts = np.append(error_counts, np.array(errors))
    name_total = y_test[y_test == name].shape[0]
    name_counts = np.append(name_counts, np.array(name_total))
display(misclassified.unique(), misclassified.shape, sum)

print(error_counts / name_counts)
error_percentage = 100 * error_counts / name_counts



# Plot misclassifications
plt.figure(figsize=(15, 5))
plt.bar(range(error_counts.size), height=error_counts, align='edge', width=0.2)
plt.xticks(range(error_counts.size), misclassified.unique(), rotation=70)
plt.show()

plt.figure(figsize=(15, 5))
plt.bar(range(error_counts.size), height=error_percentage, align='edge', width=0.2)
plt.xticks(range(error_counts.size), misclassified.unique(), rotation=70)
plt.show()